# 误差逆传播算法

误差逆传播算法（error BackPropagation，简称BP）算法也叫“反向传播算法”。

给定训练集 $D = \{ (\boldsymbol{x}_1, \boldsymbol{y}_1), (\boldsymbol{x}_2, \boldsymbol{y}_2), \ldots, (\boldsymbol{x}_m, \boldsymbol{y}_m) \},\ \boldsymbol{x}_i \in \mathbb{R}^d, \ \boldsymbol{y}_i \in \mathbb{R}^l$，即输入示例由d个属性描述，输出l维实值向量。下图给出了一个拥有d个输入神经元、l个输出神经元、q个隐层神经元的多层前馈网络结构，其中

- $\theta_j$ 表示输出层第j个神经元的阈值
- $\gamma_h$ 表示隐层第h个神经元的阈值
- $v_{ih}$ 表示输入层第i个神经元与隐层第h个神经元之间的连接权
- $w_{hj}$ 表示隐层第h个神经元与输出层第j个神经元之间的连接权

记

- $\alpha_h = \sum_{i=1}^{d} v_{ih}x_i$ 表示隐层第h个神经元接收到的输入
- $\beta_j = \sum{h=1}^{q} w_{hj}b_h$ 表示输出层第j个神经元接收到的输入，其中 $b_h$ 为隐层第h个神经元的输出。

假设隐层和输出层的激活函数都使用Sigmoid函数 $f = \frac{1}{1 + e^{-x}}$ 。

<img src="./bp.jpg" width="480" />

## 标准BP算法

对训练例 ($\boldsymbol{x}_k$, $\boldsymbol{y}_k$)，假定神经网络的输出为 $\hat{\boldsymbol{y}}_k = (\hat{y}_1^k, \hat{y}_2^k, \ldots, \hat{y}_l^k)$，即

$$ \hat{y}_j^k = f(\beta_j - \theta_j) $$

则网络在 ($\boldsymbol{x}_k$, $\boldsymbol{y}_k$) 上的均方误差为

$$ E_k = \frac{1}{2} \sum_{j=1}^{l} \left( \hat{y}_j^k - y_j^k \right)^2 $$

我们的目的就是最小化上面的均方误差。

BP算法基于梯度下降（gradient descent）策略，以目标的负梯度方向对参数进行调整：初始时随机选择所有参数，然后根据梯度下降方法对参数进行更新，直到满足一定的停止条件。我们要求解的参数有 $v_{ih}$（d×q个）、$w_{hj}$（q×l个）、$\gamma_h$（q
个）、$\theta_j$（l个），任意参数v的更新公式为

$$ v \leftarrow v + \Delta v $$

下面以 $w_{hj}$ 为例进行推导。根据梯度下降，有

$$ \Delta w_{hj} = - \eta \frac{\partial E_k}{\partial w_{hj}} $$

注意到 $w_{hj}$ 先影响到第j个输出层神经元的输入值 $\beta_j$，再影响到其输出值 $\hat{y}_j^k$，然后影响到 $E_k$，有

$$ \frac{\partial E_k}{\partial w_{hj}} = \frac{\partial E_k}{\partial \hat{y}_j^k} \cdot \frac{\partial \hat{y}_j^k}{\partial \beta_j} \cdot \frac{\partial \beta_j}{\partial w_{hj}}$$

首先，需要引入Sigmoid函数的一个性质：

$$ f^{\prime}(x) = f(x)(1-f(x)) $$

于是

$$ \begin{align*} 
\frac{\partial E_k}{\partial \hat{y}_j^k} &= \hat{y}_j^k - y_{j}^{k} \\
\frac{\partial \hat{y}_j^k}{\partial \beta_j} &= f^{\prime}(\beta_j - \theta_j) \\
&= f(\beta_j - \theta_j)(1-f(\beta_j - \theta_j)) \\
&= \hat{y}_j^k (1-\hat{y}_j^k) \\
\frac{\partial \beta_j}{\partial w_{hj}} &= b_h
\end{align*}$$

将上面的式子带入更新公式，得到

$$ \Delta w{hj} = \eta \hat{y}_j^k (1-\hat{y}_j^k) (y_{j}^{k} - \hat{y}_j^k) b_h $$

令 $g_j = \hat{y}_j^k (1-\hat{y}_j^k) (y_{j}^{k} - \hat{y}_j^k)$，则

$$ \Delta w_{hj} = \eta g_j b_h $$

类似可得

$$ \begin{align*} 
\Delta \theta_j &= - \eta g_j \\
\Delta v_{ih} &= \eta e_h x_i \\
\Delta \gamma_h &= - \eta e_h
\end{align*} $$

其中

$$ \begin{align*} 
e_h &= - \frac{\partial E_k}{\partial b_h} \cdot \frac{\partial b_h}{\partial \alpha_h} \\
&= - \sum_{j=1}^{q} \frac{\partial E_k}{\partial \beta_j} \cdot \frac{\partial \beta_j}{\partial b_h} f^{\prime}(\alpha_h - \gamma_h) \\
&= \sum_{j=1}^{l} w_{hj} g_j f^{\prime}(\alpha_h - \gamma_h) \\
&= b_h (1 - b_h) \sum_{j=1}^{l} w_{hj}g_j
\end{align*}$$

学习率 $\eta \in (0, 1)$ 控制着算法每一轮迭代中的更新步长，若太大则容易振荡，太小则收敛速度又会过慢。有事后为了做精细调节，可令 $ \Delta w_{hj} $ 与 $\Delta \theta_j$ 使用 $\eta_1$，$\Delta v_{ih} $ 与 $\Delta \gamma_h$ 使用 $\eta_2$，两者未必相等。

下图给出了BP算法的工作流程。对每个训练样例，BP算法执行以下操作：先将输入示例提供给输入层神经元，然后逐层将信号前传，直到产生输出层的结果；然后计算输出层的误差（第4-5行），再将误差逆向传播至隐层神经元（第6行），最后根据隐层神经元的误差来对连接权和阈值进行调整（第7行）。该迭代过程循环进行，直到达到某些停止条件为止，例如训练误差已达到一个很小的值。

<img src="./bp2.jpg" width="480" />

## 累积误差逆传播算法

需注意的是，BP算法的目标是要最小化训练集D上的累积误差

$$ E=\frac{1}{m}\sum_{k=1}^{m} E_k $$

但上面介绍的“标准BP算法”每次仅针对一个训练样例更新链接权和阈值，也就是说，标准BP算法的更新规则是基于单个的E_k推导而得。如果类似地推导出基于累积误差最小化的更新规则，就得到了累积误差逆传播（accumulated error backpropagation）算法。

累积BP算法与标准BP算法都很常用：

- 标准BP算法每次更新只针对单个样例，参数更新的非常频繁
- 标准BP算法对不同样例进行更新的效果可能出现“抵消”的现象
- 累积BP算法读取整个训练集D一遍后才对参数进行更新，参数更新的频率低得多
- 为了达到同样的累积误差极小点，标准BP算法往往需进行更多次数的迭代
- 很多任务中，累积误差下降到一定程度之后，进一步下降会非常缓慢，这是标准BP往往会更快获得较好的解，尤其在训练集D非常大时更明显。

> 标准BP算法和累积BP算法的区别类似于随机梯度下降与标准梯度下降之间的区别

## 实际问题

### 隐层神经元个数

Hornik等人证明，只需一个包含足够多神经元的隐层，多层前馈网络就能以任意精度逼近任意复杂度的连续函数。然而，如何设置隐层神经元的个数仍是个未决问题，实际应用中通常考“试错法”（trial-by-error）调整。

### 过拟合

由于强大的表示能力，BP神经网络经常遭遇过拟合，训练误差持续降低，但测试误差却可能上升。有两种策略缓解：

- 早停（early stopping）：将数据分成训练集和验证集，训练集用来计算梯度、更新连接权和阈值，验证集用来估计误差，若训练集误差降低但验证集误差升高，则停止训练，同时返回具有最小验证集误差的连接权和阈值。
- 正则化（regularization）：在误差目标函数中增加一个用于描述网络复杂度的部分，例如连接权与阈值的平方和。仍令 $E_k$ 表示第k个训练样本上的误差，$w_i$ 表示连接权和阈值，则误差目标函数改变为

$$ E = \lambda \frac{1}{m} \sum_{k=1}^{m} E_k + (1 - \lambda) \sum_{i}w_i^2 $$

其中 $\lambda \in (0, 1)$ 用于对经验误差与网络复杂度这两项进行折中，常通过交叉验证法来估计。